In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv
/kaggle/input/bank-customer-churn-prediction/Churn_Modelling.csv


In [ ]:
!pip install -U autogluon > /dev/null

from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [3]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
df_train.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [4]:
df_test = pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')
df_test_ov = df_test.copy()
df_test.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [5]:
df_orig=pd.read_csv("/kaggle/input/bank-customer-churn-prediction/Churn_Modelling.csv")
df_orig=df_orig.rename(columns={'RowNumber':'id'})
df_orig.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


# Preprocessing

In [6]:
# 진짜 의미가 있는 숫자형 데이터만 MinMaxScaler를 사용함.
# MinMaxScaler와 같은 작동방식으로 사용한다.

scale_cols = ['Age','CreditScore', 'Balance','EstimatedSalary']

for c in scale_cols:
    min_value = df_train[c].min()
    max_value = df_train[c].max()
    df_train[c+"_scaled"] = (df_train[c] - min_value) / (max_value - min_value)
    df_test[c+"_scaled"] = (df_test[c] - min_value) / (max_value - min_value)

In [7]:
df_train.head(2)

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Age_scaled,CreditScore_scaled,Balance_scaled,EstimatedSalary_scaled
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.0,2,1.0,0.0,181449.97,0,0.202703,0.636,0.0,0.907279
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.0,2,1.0,1.0,49503.50,0,0.202703,0.554,0.0,0.247483


In [8]:
# train과 test를 합친 다음 인덱스 제거
# 이후 주요 통계 지표 확인

df_all = pd.concat([df_train,df_test]).reset_index(drop=True)
aggs = {
    'Age': ['min','max', 'mean'],       
    'Balance': ['min','max', 'mean','sum'],
    'NumOfProducts': ['mean','sum'],
    'IsActiveMember': ['min','max', 'mean','sum'],
    'CreditScore': ['min','max', 'mean'],
    'EstimatedSalary': ['min','max', 'mean','sum'],
    'id': 'count',
}
df_grps=df_all.groupby(['CustomerId', 'Surname', 'Geography', 'Gender']).agg(aggs).reset_index()
df_grps.columns = list(map(''.join, df_grps.columns.values))
print(len(df_grps))
df_grps.head()

235258


,CustomerId,Surname,Geography,Gender,Agemin,Agemax,Agemean,Balancemin,Balancemax,Balancemean,...,IsActiveMembermean,IsActiveMembersum,CreditScoremin,CreditScoremax,CreditScoremean,EstimatedSalarymin,EstimatedSalarymax,EstimatedSalarymean,EstimatedSalarysum,idcount
0,15565701,Bruno,Spain,Male,49.0,49.0,49.0,0.00,0.00,0.00,...,1.0,1.0,846,846,846.0,90280.70,90280.70,90280.70,90280.70,1
1,15565701,Chineze,France,Male,32.0,32.0,32.0,107209.58,107209.58,107209.58,...,1.0,1.0,705,705,705.0,129822.79,129822.79,129822.79,129822.79,1
2,15565701,Ferri,France,Female,39.0,39.0,39.0,161993.89,161993.89,161993.89,...,0.0,0.0,698,698,698.0,90212.38,90212.38,90212.38,90212.38,1
3,15565701,Ferri,Spain,Male,34.0,34.0,34.0,161993.89,161993.89,161993.89,...,0.0,0.0,707,707,707.0,94912.78,94912.78,94912.78,94912.78,1
4,15565706,Akobundu,France,Male,37.0,40.0,38.5,0.00,0.00,0.00,...,0.5,1.0,612,682,647.0,83256.26,83256.26,83256.26,166512.52,2


In [9]:
aggs = {   
    'Balance': ['min','max', 'mean','sum'],
    'NumOfProducts': ['mean','sum'],
    'IsActiveMember': ['min','max', 'mean','sum'],
    'CreditScore': ['min','max', 'mean'],
    'EstimatedSalary': ['min','max', 'mean','sum'],
    'id': 'count',
}
df_grps1=df_all.groupby(['CustomerId', 'Surname', 'Age', 'Gender']).agg(aggs).reset_index()
df_grps1.columns = list(map('grps1'.join, df_grps1.columns.values))
print(len(df_grps1))
df_grps1=df_grps1.rename(columns={'CustomerIdgrps1':'CustomerId','Surnamegrps1':'Surname',
                                 'Agegrps1':'Age','Gendergrps1':'Gender'})
df_grps1.head()

252537


,CustomerId,Surname,Age,Gender,Balancegrps1min,Balancegrps1max,Balancegrps1mean,Balancegrps1sum,NumOfProductsgrps1mean,NumOfProductsgrps1sum,...,IsActiveMembergrps1mean,IsActiveMembergrps1sum,CreditScoregrps1min,CreditScoregrps1max,CreditScoregrps1mean,EstimatedSalarygrps1min,EstimatedSalarygrps1max,EstimatedSalarygrps1mean,EstimatedSalarygrps1sum,idgrps1count
0,15565701,Bruno,49.0,Male,0.00,0.00,0.00,0.00,2.0,2,...,1.0,1.0,846,846,846.0,90280.70,90280.70,90280.70,90280.70,1
1,15565701,Chineze,32.0,Male,107209.58,107209.58,107209.58,107209.58,1.0,1,...,1.0,1.0,705,705,705.0,129822.79,129822.79,129822.79,129822.79,1
2,15565701,Ferri,34.0,Male,161993.89,161993.89,161993.89,161993.89,1.0,1,...,0.0,0.0,707,707,707.0,94912.78,94912.78,94912.78,94912.78,1
3,15565701,Ferri,39.0,Female,161993.89,161993.89,161993.89,161993.89,1.0,1,...,0.0,0.0,698,698,698.0,90212.38,90212.38,90212.38,90212.38,1
4,15565706,Akobundu,35.0,Male,0.00,0.00,0.00,0.00,1.5,3,...,0.5,1.0,663,787,725.0,83256.26,83256.26,83256.26,166512.52,2


In [10]:
exitGrpBy=['CustomerId', 'Surname',  'Gender','Geography','EstimatedSalary']

exitSrtBy=['CustomerId', 'Surname',  'Gender','Geography',
       'Age', 'Tenure']
##
df_all_Exits=df_all.copy()
df_all_Exits['Exited']=df_all_Exits['Exited'].fillna(-1)
df_all_Exits=df_all_Exits.sort_values(exitSrtBy)
df_all_Exits['Exit_lag1']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(1)
df_all_Exits['Exit_lag2']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(2)
df_all_Exits['Exit_lag3']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(3)

df_all_Exits['Exit_lead1']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(-1)
df_all_Exits['Exit_lead2']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(-2)
df_all_Exits['Exit_lead3']=df_all_Exits.groupby(exitGrpBy)['Exited'].shift(-3)

df_all_Exits['Balance_lag_diff1']=df_all_Exits['Balance'].shift(1)
df_all_Exits['Balance_lead_diff1']=df_all_Exits['Balance'].shift(-1)

df_all_Exits=df_all_Exits[['id','Exit_lag1','Exit_lag2','Exit_lag3',
                         'Exit_lead1','Exit_lead2','Exit_lead3',
                          'Balance_lag_diff1','Balance_lead_diff1']]
df_all_Exits=df_all_Exits.fillna(-1).astype('int')
df_all_Exits

,id,Exit_lag1,Exit_lag2,Exit_lag3,Exit_lead1,Exit_lead2,Exit_lead3,Balance_lag_diff1,Balance_lead_diff1
87277,87277,-1,-1,-1,-1,-1,-1,-1,107209
140882,140882,-1,-1,-1,-1,-1,-1,0,161993
160702,160702,-1,-1,-1,-1,-1,-1,107209,161993
259273,259273,-1,-1,-1,-1,-1,-1,161993,0
106193,106193,-1,-1,-1,-1,-1,-1,161993,0
...,...,...,...,...,...,...,...,...,...
189759,189759,-1,-1,-1,-1,-1,-1,0,0
124621,124621,-1,-1,-1,-1,-1,-1,0,135277
97684,97684,-1,-1,-1,-1,-1,-1,0,144591
46972,46972,-1,-1,-1,-1,-1,-1,135277,0


In [11]:
def getGrpsIndv(df_orig,df_train,df_test,grpCols,nm):
    grpBy=[]
    for c in grpCols:
        for i in grpCols:
            if c!=i:
                grpBy=[c,i]
                grpBy=[c,i]
                df_tmp=df_orig.groupby(grpBy).agg({'id':'count','Exited':{'mean'}}).reset_index()
                df_tmp.columns=list(map(''.join, (list(df_tmp.columns.values))))
                sepCols=df_tmp.columns.drop(grpBy)+nm+'_ind_'+c+'-'+i
                df_tmp.columns=list(grpBy)+list(sepCols)
                #
                df_train=df_train.merge(df_tmp,how='left')
                df_test=df_test.merge(df_tmp,how='left')
                
                df_train[sepCols]=df_train[sepCols].fillna(0)
                df_test[sepCols]=df_test[sepCols].fillna(0)

                df_train[sepCols]=df_train[sepCols].astype('int')
                df_test[sepCols]=df_test[sepCols].astype('int')
    return df_train,df_test

In [12]:
def getGrps(df_orig,df_train,df_test,grpCols,nm):
    grpBy=[]
    for c in grpCols:
        grpBy.append(c)
        df_tmp=df_orig.groupby(grpBy).agg({'id':'count','Exited':{'sum'}}).reset_index()
        df_tmp.columns=list(map(''.join, (list(df_tmp.columns.values))))
        sepCols=df_tmp.columns.drop(grpBy)+nm+'_grps_'+c
        df_tmp.columns=list(grpBy)+list(sepCols)
                #
        df_train=df_train.merge(df_tmp,how='left')
        df_test=df_test.merge(df_tmp,how='left')
        
        df_train[sepCols]=df_train[sepCols].fillna(0)
        df_test[sepCols]=df_test[sepCols].fillna(0)
        
        df_train[sepCols]=df_train[sepCols].astype('int')
        df_test[sepCols]=df_test[sepCols].astype('int')
    return df_train,df_test

In [13]:
grpCols=['CustomerId', 'Surname', 'Geography', 'Gender', 'Age', 'Tenure', 'CreditScore', 
         'NumOfProducts','HasCrCard', 
         'IsActiveMember' ,'EstimatedSalary','Balance']
df_train,df_test=getGrps(df_orig,df_train,df_test,grpCols,'Orig_groups')
df_train,df_test=getGrpsIndv(df_orig,df_train,df_test,grpCols,'Orig_ind')

In [14]:
df_train.shape

(165034, 306)

In [15]:
df_train.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,...,ididOrig_ind_ind_Balance-CreditScore,ExitedmeanOrig_ind_ind_Balance-CreditScore,ididOrig_ind_ind_Balance-NumOfProducts,ExitedmeanOrig_ind_ind_Balance-NumOfProducts,ididOrig_ind_ind_Balance-HasCrCard,ExitedmeanOrig_ind_ind_Balance-HasCrCard,ididOrig_ind_ind_Balance-IsActiveMember,ExitedmeanOrig_ind_ind_Balance-IsActiveMember,ididOrig_ind_ind_Balance-EstimatedSalary,ExitedmeanOrig_ind_ind_Balance-EstimatedSalary
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,...,10,0,2600,0,2592,0,1744,0,0,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,...,15,0,2600,0,2592,0,1873,0,0,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,...,26,0,2600,0,2592,0,1744,0,0,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,...,0,0,0,0,0,0,0,0,0,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,...,11,0,2600,0,2592,0,1873,0,0,0


## Feature Engineering
- 기존 변수들을 활용한 신구 파생변수 생성
    - 장년층 여부, 나이 카테고리화, 신용카드와 활성사용자 여부 등

In [16]:
def getFeats(df):
    
    df['IsSenior'] = df['Age'].apply(lambda x: 1 if x >= 60 else 0)
    df['IsActive_by_CreditCard'] = df['HasCrCard'] * df['IsActiveMember']
    df['Products_Per_Tenure'] =  df['Tenure'] / df['NumOfProducts']
    df['AgeCat'] = np.round(df.Age/20).astype('int').astype('category')
    df['Sur_Geo_Gend_Sal'] = df['Surname']+df['Geography']+df['Gender']+np.round(df.EstimatedSalary).astype('str')
    
    df = df.merge(df_grps,how='left',on=['CustomerId', 'Surname', 'Geography', 'Gender'])
    df = df.merge(df_grps1,how='left',on=['CustomerId', 'Surname', 'Age', 'Gender'])
    df = df.merge(df_all_Exits,how='left')
    return df

In [17]:
df_train = getFeats(df_train)
df_test = getFeats(df_test)
##
feat_cols=df_train.columns.drop(['id','Exited'])
feat_cols=feat_cols.drop(scale_cols) #아예 기존 연속형 데이터는 전부 제거 -> 이미 전처리를 통해 모두 스케일링 / 범주화 / 통계화 시킴.
print("Number of Features:",len(feat_cols))
print(feat_cols)
df_train.head()

Number of Features: 352
Index(['CustomerId', 'Surname', 'Geography', 'Gender', 'Tenure',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Age_scaled',
       'CreditScore_scaled',
       ...
       'EstimatedSalarygrps1sum', 'idgrps1count', 'Exit_lag1', 'Exit_lag2',
       'Exit_lag3', 'Exit_lead1', 'Exit_lead2', 'Exit_lead3',
       'Balance_lag_diff1', 'Balance_lead_diff1'],
      dtype='object', length=352)


,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,...,EstimatedSalarygrps1sum,idgrps1count,Exit_lag1,Exit_lag2,Exit_lag3,Exit_lead1,Exit_lead2,Exit_lead3,Balance_lag_diff1,Balance_lead_diff1
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,...,181449.97,1,-1,-1,-1,-1,-1,-1,0,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,...,49503.50,1,-1,-1,-1,-1,-1,-1,115587,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,...,184866.69,1,-1,-1,-1,-1,-1,-1,0,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,...,84560.88,1,-1,-1,-1,-1,-1,-1,0,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,...,15068.83,1,-1,-1,-1,-1,-1,-1,135759,142084


In [18]:
X=df_train[feat_cols]
y=df_train['Exited']
##
cat_features = np.where(X.dtypes != np.float64)[0]
cat_features

array([  0,   1,   2,   3,   4,   5,  12,  13,  14,  15,  16,  17,  18,
        19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 18

In [19]:
X.head(3)

,CustomerId,Surname,Geography,Gender,Tenure,NumOfProducts,HasCrCard,IsActiveMember,Age_scaled,CreditScore_scaled,...,EstimatedSalarygrps1sum,idgrps1count,Exit_lag1,Exit_lag2,Exit_lag3,Exit_lead1,Exit_lead2,Exit_lead3,Balance_lag_diff1,Balance_lead_diff1
0,15674932,Okwudilichukwu,France,Male,3,2,1.0,0.0,0.202703,0.636,...,181449.97,1,-1,-1,-1,-1,-1,-1,0,0
1,15749177,Okwudiliolisa,France,Male,1,2,1.0,1.0,0.202703,0.554,...,49503.50,1,-1,-1,-1,-1,-1,-1,115587,0
2,15694510,Hsueh,France,Male,10,2,1.0,0.0,0.297297,0.656,...,184866.69,1,-1,-1,-1,-1,-1,-1,0,0


In [20]:
df_train['Exited']

0         0
1         0
2         0
3         0
4         0
         ..
165029    0
165030    0
165031    0
165032    0
165033    1
Name: Exited, Length: 165034, dtype: int64

# AutoGluon Test

ERROR: Could not find a version that satisfies the requirement autogluon (from versions: none)
ERROR: No matching distribution found for autogluon


ModuleNotFoundError: No module named 'autogluon'

In [ ]:
df_train_AG = df_train.copy()
df_train_AG.pop('id')
df_train_AG.head(3)

In [ ]:
df_test_AG = df_test.copy()
df_test_AG.pop('id')
df_test_AG.head(3)

In [ ]:
automl = TabularPredictor(
    label='Exited',
    problem_type='binary',
    eval_metric='roc_auc')

In [ ]:
automl.fit(df_train_AG, presets='best_quality')
automl.leaderboard()

In [ ]:
predictions = automl.predict_proba(df_test)
predictions0 = predictions[1]

In [ ]:
output_sample = pd.read_csv('data/sample_submission.csv')
output = pd.DataFrame({'id': output_sample.id, 'Exited': predictions0})
output.to_csv('submission.csv', index=False, sep=',')